# MidiBERT-Piano Playground

## Introduction 

#### 🎉 This is MIDI-BERT-2 playground.
This software can arrange music automatically based on user-input melody MIDI files and styles, along with a command-line interface. 

 You can try run our customized pre-trained dataset. Please Input some `.MIDI` file into the algorithm. We will Output 


1.   Key estimation (can also be specified explicitly by the user)
2.   Chord progression estimation (can also be specified explicitly by the user)
3.   Perform music editing based on the styles

```
Disclaimer:
This is a university student project (Computer Music Development in CUHK)
and it is mainly for learning purpose. 
Therefore, it is very basic and not a production-ready tool for professionals

```

# Task1. Data Collection


## 1. Download Midi File form [Live Orchestral Piano](https://qsdfo.github.io/LOP/)

In [ ]:
!wget https://qsdfo.github.io/LOP/database/SOD.zip # Get Midi Data from qsdfo
!unzip SOD.zip 

## 2. Sort 2/4, 3/4, 4/4 Time Signature 


In [ ]:
import re
from music21 import converter,instrument,meter,stream # or import *
import os
import csv  

TARGET = ['2/4','3/4','4/4']
DEBUG = False

music_2_4 = []
music_3_4 = []
music_4_4 = []

def write_csv(open_file, data):
    s = open_file+".csv"
    with open(s, 'w', encoding='UTF8') as f:
        writer = csv.writer(f)

        # write the header
        writer.writerow(["path"])

        # write the data
        writer.writerows(data)

def find_files(reg, search_path):
    result = []
    # Wlaking top-down from the root
    for root, dir, files in os.walk(search_path):
        x = re.compile(reg)
        newlist = list(filter(x.match, files)) # Read Note below
        PATH = ""
        for x in (newlist):
            PATH = "{}/{}".format(root,x)         
        if PATH:
            Time_signature(str(PATH), x)
    return result


def Time_signature(path , x):
    try:
      s = converter.parse(path)
      cnt = []
      res = []
      ft = s[stream.Part][0]
      time_signatures = ft[meter.TimeSignature]
      isBreak = True
      print("************************************start {}".format(list(time_signatures)))
      for time_signature in list(time_signatures):
        time_signature = str(time_signature)
        print(time_signature)

        if time_signature.find('TimeSignature 2/4>') > 0:
            cnt.append(1)
            
        elif  time_signature.find('TimeSignature 3/4>') > 0:
            cnt.append(1)
            
        elif  time_signature.find('TimeSignature 4/4>') > 0:
            cnt.append(1)

      print("999_cnt:{} len:{}".format(len(cnt), len(time_signatures)))
      if len(cnt) != len(time_signatures):
        print("02_--------->>>>>>>break>>>>>>> {}".format(list(time_signatures)))
        return     

      for time_signature in list(time_signatures):
          time_signature = str(time_signature)
          row = [path]
          print("--->>>>target timesignature: {}".format(time_signature))
          if '2/4' in time_signature:
              print("2/4 path:{}". format(row))
              music_2_4.append(row)
          elif '3/4' in time_signature:
              print("3/4 path:{}". format(row))
              music_3_4.append(row)
          elif '4/4' in time_signature:
              print("4/4 path:{}". format(row))
              music_4_4.append(row)
      return res
    except:
      print("fail"+ path)


if __name__ == '__main__':
    try:
      find_files('.*mid$', '/content/SOD/OpenMusicScores') 
    except:
      print("fail")
    print(music_2_4)
    print(music_3_4)
    print(music_4_4)
    write_csv("2_4",music_2_4)
    write_csv("3_4",music_3_4)
    write_csv("4_4",music_4_4)


## 3. Process [2/4, 3/4, 4/4] Time Signature with .CSV  



In [ ]:
!while read -r file; do cp "$file" /content/data ; done < music_2_4.csv
!while read -r file; do cp "$file" /content/data ; done < music_3_4.csv
!while read -r file; do cp "$file" /content/data ; done < music_4_4.csv


## 4. If you think it is too waste time. Feel free to download those sorted Data from our [Github](https://github.com/CUHK-CMD/Time-Signature-Detection/tree/main/data/ALLINONEdata)

In [3]:
!wget https://github.com/CUHK-CMD/Time-Signature-Detection/raw/main/data/dataa.zip
!unzip dataa.zip 

--2022-08-16 13:17:47--  https://github.com/CUHK-CMD/Time-Signature-Detection/raw/main/data/dataa.zip
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2022-08-16 13:17:47 ERROR 404: Not Found.

unzip:  cannot find or open dataa.zip, dataa.zip.zip or dataa.zip.ZIP.


# Task 2. Data Tokenization and Preprocessing

## Data Tokenization
  we are using [finetuning-stable](https://github.com/CUHK-CMD/MIDI-BERT-2/blob/finetuning-stable/dict/make_dict.py) for token definition and generate new pickle file

  

### install requirements

In [7]:
!pip install -r ../../requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 38.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 37.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.1/51.1 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.8/133.8 MB 22.9 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 33.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 28.8 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 30.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 613.4/613.4 kB 21.5 MB/s eta 0:00:00
  Using cached tqdm-4.64.0-py2.py3-none-any.whl (78 kB)
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 MB 28.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.0/307.0 kB 10.6 MB/s eta 0:00

In [5]:
#dict/make_dict.py
import pickle

event2word = {
    "Bar": {},
    "Position": {},
    "Pitch": {},
    "Duration": {},
    "Program": {},
    "Time Signature": {},
}
word2event = {
    "Bar": {},
    "Position": {},
    "Pitch": {},
    "Duration": {},
    "Program": {},
    "Time Signature": {},
}


def special_tok(cnt, cls):
    # """event2word[cls][cls+' <SOS>'] = cnt
    # word2event[cls][cnt] = cls+' <SOS>'
    # cnt += 1

    event2word[cls][cls + " <PAD>"] = cnt
    word2event[cls][cnt] = cls + " <PAD>"
    cnt += 1

    event2word[cls][cls + " <MASK>"] = cnt
    word2event[cls][cnt] = cls + " <MASK>"
    cnt += 1

    event2word[cls][cls + " <EOS>"] = cnt
    word2event[cls][cnt] = cls + " <EOS>"
    cnt += 1

    event2word[cls][cls + " <ABS>"] = cnt
    word2event[cls][cnt] = cls + " <ABS>"
    cnt += 1

    event2word[cls][cls + " <BOS1>"] = cnt  # string quartet 1: violin
    word2event[cls][cnt] = cls + " <BOS1>"
    cnt += 1

    event2word[cls][cls + " <BOS2>"] = cnt  # string quartet 2: violin
    word2event[cls][cnt] = cls + " <BOS2>"
    cnt += 1

    event2word[cls][cls + " <BOS3>"] = cnt  # string quartet 3: viola
    word2event[cls][cnt] = cls + " <BOS3>"
    cnt += 1

    event2word[cls][cls + " <BOS4>"] = cnt  # string quartet 4: cello
    word2event[cls][cnt] = cls + " <BOS4>"
    cnt += 1


# Bar
cnt, cls = 0, "Bar"
event2word[cls]["Bar New"] = cnt
word2event[cls][cnt] = "Bar New"
cnt += 1

event2word[cls]["Bar Continue"] = cnt
word2event[cls][cnt] = "Bar Continue"
cnt += 1
special_tok(cnt, cls)

# =============================================
# Position
# 16 beats -> 24 beats -> 48 beats
cnt, cls = 0, "Position"
for i in range(1, 49):
    event2word[cls][f"Position {i}/48"] = cnt
    word2event[cls][cnt] = f"Position {i}/48"
    cnt += 1

special_tok(cnt, cls)
# =============================================

# Note On
cnt, cls = 0, "Pitch"
for i in range(22, 108):
    event2word[cls][f"Pitch {i}"] = cnt
    word2event[cls][cnt] = f"Pitch {i}"
    cnt += 1

special_tok(cnt, cls)

# Note Duration
cnt, cls = 0, "Duration"
for i in range(12 * 4 * 4):  # MAX 4 bars
    event2word[cls][f"Duration {i}"] = cnt
    word2event[cls][cnt] = f"Duration {i}"
    cnt += 1

special_tok(cnt, cls)

# =============================================
# Program
cnt, cls = 0, "Program"
for i in range(97):  # ignore sound effects and ethnic instruments
    # The value 96 reserved for a special input in the finetuning stage.
    event2word[cls][f"Program {i}"] = cnt
    word2event[cls][cnt] = f"Program {i}"
    cnt += 1

special_tok(cnt, cls)
# =============================================


# =============================================
## Customized Tokens, summer 2022
# Simple Time Signature
supported_time_signature = ["24", "34", "44"]

cnt, cls = 0, "Time Signature"
for i in supported_time_signature:
    event2word[cls][f"Time Signature {i}"] = cnt
    word2event[cls][cnt] = f"Time Signature {i}"
    cnt += 1

special_tok(cnt, cls)
# =============================================


# print(event2word)
# print(word2event)
t = (event2word, word2event)

with open("../../dict/CP_program.pkl", "wb") as f:
    pickle.dump(t, f)

## Now you get Data Tokenization

- Then, we do the Preprocessing
  - we got string quartet preprocessing and multiprocessing
  - for string quartet preprocessing
    - please look at `CP/main.py`  
  - for multiprocessing
    - please look at `CP_string/main.py` 

###  Pretraining

In [23]:
### string quartet preprocessing
!python ./main.py --task skyline --input_dir ../../skyline_data --output_dir ../../skyline_data --name skylineNPY --dict ../../dict/CP_program.pkl


Traceback (most recent call last):
  File "/Users/icerxm/Github/MIDI-BERT-2/tutorial/cantonese/./../../CP/main.py", line 59, in <module>
    assert len(files)
AssertionError
